## Cargar librerías de todo el programa

##### Celda requerida siempre

In [1]:
import tkinter as tk                            # Para la interfaz grafica
from tkinter import *                           # Para la interfaz grafica
from tkinter import scrolledtext                # Para la interfaz grafica
import pandas as pd                             # Para manejo de dataframes
import math                                     # Para operar con logaritmos
import time
import nltk
from nltk.tokenize import RegexpTokenizer       # Para tokenizar un texto
from nltk.corpus import stopwords               # Para detectar stopwords en un texto
from nltk.stem.snowball import SnowballStemmer  # Para stemizar los tokens
import numpy as np
import matplotlib.pyplot as plt
import re

## Generar el DataFrame con vectores precalculados

##### Celda requerida siempre

In [2]:
def toStem(texto):
    tokensA = RegexpTokenizer(r'\w+').tokenize(texto.lower()) # Tokenizar y pasar a minusculas el texto recibido
    # ---------- Eliminar stop words ----------
    tokensB = [] # Lista que guardara la nueva lista de tokens sin stop words
    for tok in tokensA: # Recorrer los tokens uno a uno
        if not tok in stopwords.words("spanish"): # Si no es stop-word, añadir a la lista sin stop-words
            tokensB.append(tok)
    # ---------- Hacer stemming ----------
    tokensC = [] # Lista que guardara la nueva lista de tokens stemizados
    for tok in tokensB:# Recorrer los tokens token uno a uno
        tokensC.append(SnowballStemmer(language="spanish").stem(tok)) # Transformar cada token a su correspondiente stem
    return tokensC

def textToVector(texto, miDiccionario):
    vector = [0]*len(miDiccionario) # Crear el vector por defecto (todo 0). Ocurrira si el texto no tiene palabras validas
    toks = toStem(texto) # Tokenizar y stemizar el texto
    for tok in toks: # Recorrer los tokens, uno a uno
        if tok in miDiccionario:
            posicion = miDiccionario.index(tok) # Si el token esta en el diccionario, +1 al vector en la pos del token
            vector[posicion] +=1
    return vector

##### Celda requerida solo cuando se desee resetear las noticias

In [3]:
def crearDiccionario():
    diccionario = []
    for i in noticias["Titulos"]:
        for j in toStem(i): # Buscar palabras nuevas en cada titulo en particular
            if not j in diccionario: # Si la palabra seleccionada no esta ya en el diccionario, añadirla
                diccionario.append(j)
    for i in noticias["Entradillas"]:
        for j in toStem(i): # Buscar palabras nuevas en cada entradilla en particular
            if not j in diccionario: # Si la palabra seleccionada no esta ya en el diccionario, añadirla
                diccionario.append(j)
    for i in noticias["Textos"]:
        for j in toStem(i): # Buscar palabras nuevas en cada texto en particular
            if not j in diccionario: # Si la palabra seleccionada no esta ya en el diccionario, añadirla
                diccionario.append(j)
    for i in noticias["Etiquetas"]:
        for j in toStem(i): # Buscar palabras nuevas en cada etiqueta en particular
            if not j in diccionario: # Si la palabra seleccionada no esta ya en el diccionario, añadirla
                diccionario.append(j)
    return diccionario

def columnasVectores(col1, col2, col3, col4):
    superLista = []
    for i in col1:
        superLista.append(textToVector(i, diccionario)) # Vectoriza todos los titulos
    noticias["Titulos_vector"] = superLista #  Añade los vectores como columna nueva
    superLista = []
    for i in col2:
        superLista.append(textToVector(i, diccionario)) # Vectoriza todas las entradillas
    noticias["Entradillas_vector"] = superLista #  Añade los vectores como columna nueva
    superLista = []
    for i in col3:
        superLista.append(textToVector(i, diccionario)) # Vectoriza todos los textos
    noticias["Textos_vector"] = superLista #  Añade los vectores como columna nueva
    superLista = []
    for i in col4:
        superLista.append(textToVector(i, diccionario)) # Vectoriza todas las etiquetas
    noticias["Etiquetas_vector"] = superLista #  Añade los vectores como columna nueva
    return noticias

def toIDF():
    titulos = []
    entradillas = []
    textos = []
    etiquetas = []
    
    # ---------- Recoger todos los vectores a convertir en TF-IDF ----------
    for i in range(len(noticias)): 
        titulos.append(list(noticias.iloc[i][6]))
        entradillas.append(list(noticias.iloc[i][7]))
        textos.append(list(noticias.iloc[i][8]))
        etiquetas.append(list(noticias.iloc[i][9]))

    # ---------- Determinar la cantidad de noticias en las que aparece cada token ----------
    nuevoDiccionarioIDF = [0]*len(diccionario) # Lista que almacenara en cuantas noticias aparece cada token

    for i in range(len(titulos)): # Por cada linea, si al menos una columna (titulo, entradilla...) contiene el token, +1 al contador
        for j in range(len(titulos[0])):
            if titulos[i][j] > 0 or entradillas[i][j] > 0 or textos[i][j] > 0 or etiquetas[i][j] > 0:
                nuevoDiccionarioIDF[j] +=1
                
    # ---------- Transformar TF a TF-IDF ----------
    for i in titulos:
        for j in range(len(i)):
            i[j] = i[j] * math.log(len(noticias)/nuevoDiccionarioIDF[j]) # Aplicar formula TF-IDF de los apuntes
    for i in entradillas:
        for j in range(len(i)):
            i[j] = i[j] * math.log(len(noticias)/nuevoDiccionarioIDF[j]) # Aplicar formula TF-IDF de los apuntes
    for i in textos:
        for j in range(len(i)):
            i[j] = i[j] * math.log(len(noticias)/nuevoDiccionarioIDF[j]) # Aplicar formula TF-IDF de los apuntes
    for i in etiquetas:
        for j in range(len(i)):
            i[j] = i[j] * math.log(len(noticias)/nuevoDiccionarioIDF[j]) # Aplicar formula TF-IDF de los apuntes
    
    # ---------- Guardar vectores IDF en el dataframe ----------
    noticias["TitulosIDF"] = titulos
    noticias["EntradillasIDF"] = entradillas
    noticias["TextosIDF"] = textos
    noticias["EtiquetasIDF"] = etiquetas
    return nuevoDiccionarioIDF

start_time = time.time()

noticias = pd.read_csv("noticiarioCompleto.csv") # Cargar CSV obtenido del web scraping
noticias = noticias.drop("Unnamed: 0", axis=1).drop("Número de Noticia", axis=1).drop("Fechas", axis=1) # Eliminar columnas que no utlizamos

diccionario = crearDiccionario() # Crear diccionario con todos token encontrados en todo el df
noticias = columnasVectores(noticias["Titulos"], noticias["Entradillas"], noticias["Textos"], noticias["Etiquetas"]) # Crear columnas de vectores
diccionarioIDF = toIDF()
toIDF() # Crear columnas de TF-IDF

ids = []
for i in range(len(noticias)): # Añadir columna con IDs para facilitar la busqueda de una noticia concreta
    ids.append("%02d" % i)
noticias.insert(loc = 0, column = "ID", value = ids)

guardarDic = str(diccionario)
ficheroDic = open("diccionario.txt", "w")
ficheroDic.write(guardarDic)
ficheroDic.close()

guardarDicIDF = str(diccionarioIDF)
ficheroDicIDF = open("diccionarioIDF.txt", "w")
ficheroDicIDF.write(guardarDicIDF)
ficheroDicIDF.close()

noticias.to_csv("noticias_vectorizado.csv")

print("Tiempo requerido: %s segundos" % (round(time.time()-start_time, 2))) # Tiempo que tarda en generar el DataFrame

Tiempo requerido: 36.7 segundos


## Recuperar DataFrame de noticias y diccionario precalculados

##### A partir de aquí, todas las celdas requeridas siempre

In [4]:
noticias = pd.read_csv("noticias_vectorizado.csv") # Cargar CSV obtenido del web scraping
noticias = noticias.drop("Unnamed: 0", axis=1) # Eliminar columnas que no utlizamos

In [5]:
ficheroDic = open("diccionario.txt", "r")
dicc = ficheroDic.read().replace("[", "").replace("]", "").replace("'", "").replace(" ", "").split(",")
ficheroDic.close()

In [6]:
def stringVectorToFloatVector(miLista):
    resultado = []
    for t in miLista:
        resultado.append(float(t))
    return resultado

ficheroDicIDF = open("diccionarioIDF.txt", "r")
diccIDF = ficheroDicIDF.read().replace("[", "").replace("]", "").replace("'", "").replace(" ", "").split(",")
ficheroDicIDF.close()
diccIDF = stringVectorToFloatVector(diccIDF)
#diccIDF

## Funciones comunes en varias pantallas

In [7]:
def Leer(valor, contenedor):
    if(valor.get() != "No hay noticias"):
        posicion = valor.get()[8:10]
        texto = "TÍTULO:\n" + noticias.iloc[int(posicion)][1] + "\n"
        texto += "-"*60+"\n"
        texto += "ENTRADILLA:\n" + noticias.iloc[int(posicion)][2] + "\n"
        texto += "-"*60+"\n"
        texto += "ETIQUETAS:\n" + noticias.iloc[int(posicion)][6] + "\n"
        texto += "-"*60+"\n"
        texto += "NOTICIA:\n" + noticias.iloc[int(posicion)][3]       
        contenedor.delete('1.0', END)
        contenedor.insert(END, texto)

def similitudCoseno(array1, array2):
    #Comprobamos que ambas listas son del mismo tamaño
    if len(array1) != len(array2):
        diferenciaArrays = len(array1) - len(array2)
        for i in range(diferenciaArrays):
            array2 = np.append(array2, 0)
    numerador = 0
    valoresRaizArray1 = 0
    valoresRaizArray2 = 0
    #Fórmula Teorema Similitud del Coseno 
    for i in range(len(array1)):
        numerador += (array1[i] * array2[i])
        valoresRaizArray1 += array1[i]**2
        valoresRaizArray2 += array2[i]**2
    raizArray1 = valoresRaizArray1**(0.5)
    raizArray2 = valoresRaizArray2**(0.5)
    denominador = raizArray1 * raizArray2
    if(denominador == 0): # la palabra de la query no está en el diccionario
        resultado = 0
    else:
        resultado = numerador/denominador
    return resultado

def Aplicar(valorMostradoMedio, valorMostradoCategoria, valorMostradoNoticias, self):
    df_seleccionados = noticias[noticias["Noticiario"] == valorMostradoMedio.get().replace(" ", "")]
    df_seleccionados = df_seleccionados[df_seleccionados["Categoria"] == valorMostradoCategoria.get()]

    nombres = list()
    for i in range(len(df_seleccionados)):
        nombres.append("Noticia " + str("%02d" % df_seleccionados.iloc[i][0]))

    valorMostradoNoticias.set(nombres[0]) # Por defecto se mostrará la primera opcion encontrada
    desplegableNoticias = OptionMenu(self, valorMostradoNoticias, *nombres) # Cambiar las opciones
    desplegableNoticias.grid(row=5, column=1, padx=(0, 20), sticky="w") # Ubicar el desplegable

## Funciones de los botones de la ventana 1 - Comparación por etiquetas (Sorensen-Dice)

In [8]:
def sorensenEtiquetas(arrA, arrB): # Calcular la similitud de dos arrays mediante Sorensen-Dice   
    coincidencias = 0
    etsA = 0 # Numero de elementos del primer array
    etsB = 0 # Numero de elementos del segundo array
    for i in range(len(arrA)):
        if(arrA[i] == "1"):
            etsA += 1
        if(arrB[i] == "1"): 
            etsB += 1
        if(arrA[i] == "1" and arrB[i] == "1"):
            coincidencias += 1
    coeficiente = (2*coincidencias)/(etsA+etsB) # Formula del coeficiente de Sorensen-Dice
    return coeficiente

def BuscarSorensen(valorMostradoNoticias, valorMostradoFiltrar, valorMostradoTOPN, valorMostradoRanking, self):
    if(valorMostradoNoticias.get() != "No hay noticias"):
        medioDelOriginal = noticias.iloc[int(valorMostradoNoticias.get()[8:10])][4]
        etiquetasBase = noticias.iloc[int(valorMostradoNoticias.get()[8:10])][10] # Obtener las etiquetas de la noticia seleccionada
        # ---------- Listar todas las candidatas ----------
        if(valorMostradoFiltrar.get() == "Todos"):
            df_seleccionados = noticias.copy()
        else:
            df_seleccionados = noticias[noticias["Noticiario"] == valorMostradoFiltrar.get().replace(" ", "")].copy()     
        # ---------- Ordenar por similitud ----------
        superLista = []
        for i in df_seleccionados["Etiquetas_vector"]:
            superLista.append(sorensenEtiquetas(i.replace(" ", "").split(","), etiquetasBase.replace(" ", "").split(",")))
        df_seleccionados["Similitud"] = superLista
        df_seleccionados = df_seleccionados.sort_values(by="Similitud", ascending=False)

        opcionesRanking = []
        for i in range(int(valorMostradoTOPN.get())):
            opcionesRanking.append("Noticia " + str("%02d" % df_seleccionados.iloc[i]["ID"]) + " (" + str(round(df_seleccionados.iloc[i]["Similitud"]*100, 3)) + "%)")
        if(valorMostradoFiltrar.get().replace(" ", "") == medioDelOriginal or valorMostradoFiltrar.get() == "Todos"):
            valorMostradoRanking.set(opcionesRanking[1]) # Por defecto se mostrará y se escogerá la opción indicada en el enunciado
        else:
            valorMostradoRanking.set(opcionesRanking[0]) # Por defecto se mostrará y se escogerá la opción indicada en el enunciado
        desplegableRanking = OptionMenu(self, valorMostradoRanking, *opcionesRanking) # Crear el desplegable
        desplegableRanking.grid(row=11, column=1, padx=(0, 20), pady=(20, 20), sticky="w") # Ubicar el desplegable

## Controlador de la ventana 1

In [9]:
class PantallaSorensen(tk.Frame):
    def __init__(self, master):
        tk.Frame.__init__(self, master)
        tk.Frame.configure(self)

        labelTitulo = Label(self, text="COMPARADOR DE NOTICIAS POR SUS ETIQUETAS - MÉTODO SORENSEN DICE", bg=("#DDDDFF")).grid(row=0, column=0, columnspan=4, sticky="ew", pady=(20, 0))
        
        labelCategoriaSeleccion = Label(self, text="Seleccionar noticia de referencia:").grid(row=1, column=0, sticky="w", padx=(30, 0)) # Crear y ubicar el texto

        labelMedioSeleccion = Label(self, text="Medio:").grid(row=2, column=0, sticky="w", padx=(30, 20)) # Crear y ubicar el texto
        opcionesMedio = ["El Mundo", "El Pais", "20 Minutos"] # Opciones que se mostrarán en el desplegable del medio
        valorMostradoMedio = StringVar(self) # Para recoger el dato seleccionado
        valorMostradoMedio.set(opcionesMedio[0]) # Por defecto se mostrará y se escogerá la opción indicada en el enunciado
        desplegableMedio = OptionMenu(self, valorMostradoMedio, *opcionesMedio) # Crear el desplegable
        desplegableMedio.grid(row=2, column=1, padx=(0, 20), sticky="w") # Ubicar el desplegable

        labelCategoriaSeleccion = Label(self, text="Categoría:").grid(row=3, column=0, sticky="w", padx=(30, 20)) # Crear y ubicar el texto
        opcionesCategoria = ["Salud", "Tecnologia", "Ciencia"] # Opciones que se mostrarán en el desplegable de la categoría
        valorMostradoCategoria = StringVar(self) # Para recoger el dato seleccionado
        valorMostradoCategoria.set(opcionesCategoria[0]) # Por defecto se mostrará y se escogerá la opción indicada en el enunciado
        desplegableCategoria = OptionMenu(self, valorMostradoCategoria, *opcionesCategoria) # Crear el desplegable
        desplegableCategoria.grid(row=3, column=1, padx=(0, 20), sticky="w") # Ubicar el desplegable

        labelPreview = Label(self, text="Preview:").grid(row=1, column=2, sticky="sw", padx=20, pady=(20, 0)) # Crear y ubicar el texto
        contenedorPreview = scrolledtext.ScrolledText(self, wrap="word", font=("Comic Sans", 9), width=50, height=35, padx=5) # Crear el cuadro de texto de varias filas
        contenedorPreview.grid(row=2, column=2, rowspan=12, padx=(20, 0), pady=(0, 20)) # Ubicar el cuadro de texto de varias filas

        df_seleccionados = pd.DataFrame()

        botonAplicar = Button(self, text="Aplicar", command=lambda:Aplicar(valorMostradoMedio, valorMostradoCategoria, valorMostradoNoticias, self)).grid(row=4, column=0, padx=(30, 20), pady=(10, 10), columnspan=2, sticky="ew") # Crear y ubicar el botón

        labelNoticiaSeleccion = Label(self, text="Encontradas:").grid(row=5, column=0, sticky="w", padx=(30, 0)) # Crear y ubicar el texto
        opcionesNoticias = ["No hay noticias"] # Opciones que se mostrarán en el desplegable de las noticias
        valorMostradoNoticias = StringVar(self) # Para recoger el dato seleccionado
        valorMostradoNoticias.set(opcionesNoticias[0]) # Por defecto se mostrará y se escogerá la opción indicada en el enunciado
        desplegableNoticias = OptionMenu(self, valorMostradoNoticias, *opcionesNoticias) # Crear el desplegable
        desplegableNoticias.grid(row=5, column=1, padx=(0, 20), sticky="w") # Ubicar el desplegable

        botonLeer = Button(self, text="Leer", command=lambda:Leer(valorMostradoNoticias, contenedorPreview)).grid(row=6, column=0, padx=(30, 20), pady=(10, 10), columnspan=2, sticky="ew") # Crear y ubicar el botón

        labelTOPNSeleccion = Label(self, text="TOP-N:").grid(row=7, column=0, sticky="w", padx=(30, 20)) # Crear y ubicar el texto
        opcionesTOPN = [2, 3, 4, 5, 6, 7, 8, 9, 10] # Opciones que se mostrarán en el desplegable del TOP-N
        valorMostradoTOPN = StringVar(self) # Para recoger el dato seleccionado
        valorMostradoTOPN.set(opcionesTOPN[3]) # Por defecto se mostrará y se escogerá la opción indicada en el enunciado
        desplegableTOPN = OptionMenu(self, valorMostradoTOPN, *opcionesTOPN) # Crear el desplegable
        desplegableTOPN.grid(row=7, column=1, padx=(0, 20), sticky="w") # Ubicar el desplegable

        labelFiltrar = Label(self, text="Mostrar de:").grid(row=8, column=0, sticky="w", padx=(30, 20)) # Crear y ubicar el texto
        opcionesFiltrar = ["Todos", "El Mundo", "El Pais", "20 Minutos"] # Opciones que se mostrarán en el desplegable de Filtrar
        valorMostradoFiltrar = StringVar(self) # Para recoger el dato seleccionado
        valorMostradoFiltrar.set(opcionesFiltrar[0]) # Por defecto se mostrará y se escogerá la opción indicada en el enunciado
        desplegableFiltrar = OptionMenu(self, valorMostradoFiltrar, *opcionesFiltrar) # Crear el desplegable
        desplegableFiltrar.grid(row=8, column=1, padx=(0, 20), sticky="w") # Ubicar el desplegable

        botonBuscar = Button(self, text="Buscar", command=lambda:BuscarSorensen(valorMostradoNoticias, valorMostradoFiltrar, valorMostradoTOPN, valorMostradoRanking, self)).grid(row=9, column=0, padx=(30, 20), pady=(10, 10), columnspan=2, sticky="ew") # Crear y ubicar el botón

        labelSeleccion = Label(self, text="Seleccionar noticia para leer:").grid(row=10,column=0, columnspan=4, sticky="w", padx=30, pady=(20, 20)) # Crear y ubicar el texto

        labelRanking = Label(self, text="Ranking:").grid(row=11, column=0, padx=(20, 0), pady=(20, 20), sticky="e") # Crear y ubicar el texto
        opcionesRanking = ["No hay noticias"] # Opciones que se mostrarán en el desplegable del ranking
        valorMostradoRanking = StringVar(self) # Para recoger el dato seleccionado
        valorMostradoRanking.set("No hay noticias") # Por defecto se mostrará y se escogerá la opción indicada en el enunciado
        desplegableRanking = OptionMenu(self, valorMostradoRanking, *["No hay noticias"]) # Crear el desplegable
        desplegableRanking.grid(row=11, column=1, padx=(0, 20), pady=(20, 20), sticky="w") # Ubicar el desplegable

        botonVerRecom = Button(self, text="Ver recomendacion", command=lambda:Leer(valorMostradoRanking, contenedorNoticia)).grid(row=12, column=0, padx=(30, 20), pady=(10, 10), columnspan=2, sticky="ew") # Crear y ubicar el botón

        labelNoticia = Label(self, text="Texto de la noticia recomendada:").grid(row=1, column=3, sticky="sw", padx=20) # Crear y ubicar el texto
        contenedorNoticia = scrolledtext.ScrolledText(self, wrap="word", font=("Comic Sans", 9), width=50, height=35, padx=5) # Crear el cuadro de texto de varias filas
        contenedorNoticia.grid(row=2, column=3, rowspan=12, padx=(20, 20), pady=(0, 20)) # Ubicar el cuadro de texto de varias filas
        
        botonVolver = Button(self, text="Volver", command=lambda: master.cambiarFrame(VentanaPrincipal), bg="#DDDDFF").grid(row=13, column=0, columnspan=2, padx=(30, 20), pady=(10, 10))

## Funciones de los botones de la ventana 2 - Búsqueda por Query (Coseno)

In [10]:
def stringToVector(miString):
    miString = miString.replace(" ", "").replace("[", "").replace("]", "").split(",")
    resultado = []
    for t in miString:
        resultado.append(float(t))
    return resultado

def BuscarQuery(contenedorNoticia2, valorMostradoFiltrar, cuadroConsulta, valorMostradoRanking2, valorMostradoN, desplegableRanking, labelFeedback, self):
    labelFeedback = Label(self, text="", bg="#EEEEEE").grid(row=2, column=0, columnspan=2, padx=10, pady=10, sticky="ew") # Crear y ubicar el texto
    # Buscar las noticias que cumplan los parametros
    df_seleccionados = noticias.copy()
    if(valorMostradoFiltrar.get() != "Todos"):
        df_seleccionados = df_seleccionados[df_seleccionados["Noticiario"] == valorMostradoFiltrar.get().replace(" ", "")]
        df_seleccionados = df_seleccionados.reset_index()
    
    if(cuadroConsulta.get()!=""): # La formulas pueden dar errores si la consulta tiene 0 palabras
        similitudes = []
        vectorQuery = textToVector(cuadroConsulta.get(), dicc)
        for i in range(len(vectorQuery)):
            vectorQuery[i] = vectorQuery[i] * math.log(len(noticias)/int(diccIDF[i])) # Aplicar formula TF-IDF de los apuntes       
        for i in range(len(df_seleccionados)):
            vectorFila = df_seleccionados["TextosIDF"][i]
            vectorFila = stringToVector(vectorFila)
            similitudes.append(similitudCoseno(vectorQuery, vectorFila))
        df_seleccionados["Similitud"] = similitudes
        df_seleccionados = df_seleccionados.sort_values(by="Similitud", ascending=False)
        df_seleccionados = df_seleccionados.reset_index()
        
        rankingSimilitudes = []
        for i in range(int(valorMostradoN.get())):
            rankingSimilitudes.append(df_seleccionados["Similitud"][i])
        indexes = []
        for i in range(int(valorMostradoN.get())):
            indexes.append(df_seleccionados["ID"][i])
        for i in range(len(indexes)):
            indexes[i] = "Noticia " + str(indexes[i]) + " (" + str("{:.2f}".format(rankingSimilitudes[i]*100)) + "%)"     
        
        valorMostradoRanking2.set(indexes[0]) # Por defecto se mostrará y se escogerá la opción indicada en el enunciado
        desplegableRanking = OptionMenu(self, valorMostradoRanking2, *indexes) # Crear el desplegable
        desplegableRanking.grid(row=6, column=1, padx=(0, 20), pady=(20, 10), sticky="sw") # Ubicar el desplegable
        
        if("0.00" in valorMostradoRanking2.get()):
            labelFeedback = Label(self, text="No hay coincidencias", bg="red").grid(row=2, column=0, columnspan=2, padx=10, pady=10, sticky="ew") # Crear y ubicar el texto

## Controlador de la ventana 2

In [11]:
class PantallaQuery(tk.Frame):
    def __init__(self, master):
        tk.Frame.__init__(self, master)
        tk.Frame.configure(self)
        
        labelTitulo = Label(self, text="BÚSQUEDA DE NOTICIAS POR QUERY - MÉTODO DEL COSENO", bg=("#CCFFCC")).grid(row=0, column=0, columnspan=4, sticky="ew", pady=10)

        labelConsulta = Label(self, text="Consulta:").grid(row=1, column=0, padx=10, pady=10, sticky="e") # Crear y ubicar el texto
        cuadroConsulta = Entry(self)
        cuadroConsulta.grid(row=1, column=1, padx=(10, 30), pady=10, columnspan=2, sticky="ew") # Crear y ubicar el cuadro de texto

        labelFeedback = Label(self, text="").grid(row=2, column=0, columnspan=2, padx=10, pady=10, sticky="ew") # Crear y ubicar el texto
        
        labelTopN = Label(self, text="TOP-N:").grid(row=3, column=0, padx=(20, 0), sticky="e") # Crear y ubicar el texto
        opcionesN = [2, 3, 4, 5, 6, 7, 8, 9, 10, 20] # Opciones que se mostrarán en el desplegable de TOP-N
        valorMostradoN = StringVar(self) # Para recoger el dato seleccionado
        valorMostradoN.set(opcionesN[3]) # Por defecto se mostrará y se escogerá la opción indicada en el enunciado
        desplegableN = OptionMenu(self, valorMostradoN, *opcionesN) # Crear el desplegable
        desplegableN.grid(row=3, column=1, padx=(0, 20), sticky="w") # Ubicar el desplegable

        labelFiltrar = Label(self, text="Noticiario:").grid(row=4, column=0, padx=(20, 0), sticky="e") # Crear y ubicar el texto
        opcionesFiltrar = ["Todos", "El Mundo", "El Pais", "20 Minutos"] # Opciones que se mostrarán en el desplegable de Filtrar
        valorMostradoFiltrar = StringVar(self) # Para recoger el dato seleccionado
        valorMostradoFiltrar.set(opcionesFiltrar[0]) # Por defecto se mostrará y se escogerá la opción indicada en el enunciado
        desplegableFiltrar = OptionMenu(self, valorMostradoFiltrar, *opcionesFiltrar) # Crear el desplegable
        desplegableFiltrar.grid(row=4, column=1, padx=10, sticky="w") # Ubicar el desplegable

        botonBuscar = Button(self, text="Buscar", command=lambda:BuscarQuery(contenedorNoticia2, valorMostradoFiltrar, cuadroConsulta, valorMostradoRanking2, valorMostradoN, desplegableRanking, labelFeedback, self)).grid(row=5, column=0, columnspan=2, padx=20, pady=10, sticky="ew") # Crear y ubicar el botón

        labelRanking = Label(self, text="Ranking:").grid(row=6, column=0, padx=(20, 0), pady=10, sticky="se") # Crear y ubicar el texto
        opcionesRanking = ["No hay noticias"] # Opciones que se mostrarán en el desplegable del ranking
        valorMostradoRanking2 = StringVar(self) # Para recoger el dato seleccionado
        valorMostradoRanking2.set(opcionesRanking[0]) # Por defecto se mostrará y se escogerá la opción indicada en el enunciado
        desplegableRanking = OptionMenu(self, valorMostradoRanking2, *opcionesRanking) # Crear el desplegable
        desplegableRanking.grid(row=6, column=1, padx=(0, 20), pady=10, sticky="sw") # Ubicar el desplegable

        labelNoticia = Label(self, text="Texto de la noticia:").grid(row=2, column=2, sticky="w", padx=20, pady=10) # Crear y ubicar el texto
        contenedorNoticia2 = scrolledtext.ScrolledText(self, wrap="word", font=("Comic Sans", 9), width=90, height=20, padx=5) # Crear el cuadro de texto de varias filas
        contenedorNoticia2.grid(row=3, column=2, rowspan=7, padx=(20, 20), pady=(10, 20)) # Ubicar el cuadro de texto de varias filas
                
        botonLeer = Button(self, text="Leer noticia", command=lambda:Leer(valorMostradoRanking2, contenedorNoticia2)).grid(row=8, column=0, columnspan=2, padx=20, pady=10, sticky="ew") # Crear y ubicar el botón

        botonVolver = Button(self, text="Volver", command=lambda: master.cambiarFrame(VentanaPrincipal), bg="#CCFFCC").grid(row=9, column=0, columnspan=2, padx=(30, 20), pady=10)

## Funciones de los botones de la ventana 3 - Comparación por textos (Coseno)

In [12]:
def BuscarCoseno(valorMostradoTOPN, valorMostradoFiltrar, contenedorPreview, valorMostradoRanking3, self):
    # Buscar las noticias que cumplan los parametros
    df_seleccionados = noticias.copy()
    if(valorMostradoFiltrar.get() != "Todos"):
        df_seleccionados = df_seleccionados[df_seleccionados["Noticiario"] == valorMostradoFiltrar.get().replace(" ", "")]
        df_seleccionados = df_seleccionados.reset_index()
    
    similitudes = []
    vectorSeleccionada = textToVector(contenedorPreview.get(1.0, END).split("NOTICIA:")[1], dicc)
    for i in range(len(vectorSeleccionada)):
        vectorSeleccionada[i] = vectorSeleccionada[i] * math.log(len(noticias)/int(diccIDF[i])) # Aplicar formula TF-IDF de los apuntes
    for i in range(len(df_seleccionados)):
        vectorFila = df_seleccionados["TextosIDF"][i]
        vectorFila = stringToVector(vectorFila)
        similitudes.append(similitudCoseno(vectorSeleccionada, vectorFila))
    df_seleccionados["Similitud"] = similitudes
    df_seleccionados = df_seleccionados.sort_values(by="Similitud", ascending=False)
    df_seleccionados = df_seleccionados.reset_index()

    rankingSimilitudes = []
    for i in range(int(valorMostradoTOPN.get())):
        rankingSimilitudes.append(df_seleccionados["Similitud"][i])
    indexes = []
    for i in range(int(valorMostradoTOPN.get())):
        indexes.append(df_seleccionados["ID"][i])
    for i in range(len(indexes)):
        indexes[i] = "Noticia " + str(indexes[i]) + " (" + str("{:.2f}".format(rankingSimilitudes[i]*100)) + "%)"     

    valorMostradoRanking3.set(indexes[0]) # Por defecto se mostrará y se escogerá la opción indicada en el enunciado
    desplegableRanking = OptionMenu(self, valorMostradoRanking3, *indexes) # Crear el desplegable
    desplegableRanking.grid(row=11, column=1, padx=(0, 20), pady=(20, 20), sticky="sw") # Ubicar el desplegable

## Controlador de la ventana 3

In [13]:
class PantallaComparacion(tk.Frame):
    def __init__(self, master):
        tk.Frame.__init__(self, master)
        tk.Frame.configure(self)

        labelTitulo = Label(self, text="COMPARADOR DE NOTICIAS POR SUS TEXTOS - MÉTODO DEL COSENO", bg=("#FFFFCC")).grid(row=0, column=0, columnspan=4, sticky="ew", pady=(20, 0))
        
        labelCategoriaSeleccion = Label(self, text="Seleccionar noticia de referencia:").grid(row=1, column=0, sticky="w", padx=(30, 0)) # Crear y ubicar el texto

        labelMedioSeleccion = Label(self, text="Medio:").grid(row=2, column=0, sticky="w", padx=(30, 20)) # Crear y ubicar el texto
        opcionesMedio = ["El Mundo", "El Pais", "20 Minutos"] # Opciones que se mostrarán en el desplegable del medio
        valorMostradoMedio = StringVar(self) # Para recoger el dato seleccionado
        valorMostradoMedio.set(opcionesMedio[0]) # Por defecto se mostrará y se escogerá la opción indicada en el enunciado
        desplegableMedio = OptionMenu(self, valorMostradoMedio, *opcionesMedio) # Crear el desplegable
        desplegableMedio.grid(row=2, column=1, padx=(0, 20), sticky="w") # Ubicar el desplegable

        labelCategoriaSeleccion = Label(self, text="Categoría:").grid(row=3, column=0, sticky="w", padx=(30, 20)) # Crear y ubicar el texto
        opcionesCategoria = ["Salud", "Tecnologia", "Ciencia"] # Opciones que se mostrarán en el desplegable de la categoría
        valorMostradoCategoria = StringVar(self) # Para recoger el dato seleccionado
        valorMostradoCategoria.set(opcionesCategoria[0]) # Por defecto se mostrará y se escogerá la opción indicada en el enunciado
        desplegableCategoria = OptionMenu(self, valorMostradoCategoria, *opcionesCategoria) # Crear el desplegable
        desplegableCategoria.grid(row=3, column=1, padx=(0, 20), sticky="w") # Ubicar el desplegable

        labelPreview = Label(self, text="Preview:").grid(row=1, column=2, sticky="sw", padx=20, pady=(20, 0)) # Crear y ubicar el texto
        contenedorPreview = scrolledtext.ScrolledText(self, wrap="word", font=("Comic Sans", 9), width=50, height=35, padx=5) # Crear el cuadro de texto de varias filas
        contenedorPreview.grid(row=2, column=2, rowspan=12, padx=(20, 0), pady=(0, 20)) # Ubicar el cuadro de texto de varias filas

        df_seleccionados = pd.DataFrame()

        botonAplicar = Button(self, text="Aplicar", command=lambda:Aplicar(valorMostradoMedio, valorMostradoCategoria, valorMostradoNoticias, self)).grid(row=4, column=0, padx=(30, 20), pady=(10, 10), columnspan=2, sticky="ew") # Crear y ubicar el botón

        labelNoticiaSeleccion = Label(self, text="Encontradas:").grid(row=5, column=0, sticky="w", padx=(30, 0)) # Crear y ubicar el texto
        opcionesNoticias = ["No hay noticias"] # Opciones que se mostrarán en el desplegable de las noticias
        valorMostradoNoticias = StringVar(self) # Para recoger el dato seleccionado
        valorMostradoNoticias.set(opcionesNoticias[0]) # Por defecto se mostrará y se escogerá la opción indicada en el enunciado
        desplegableNoticias = OptionMenu(self, valorMostradoNoticias, *opcionesNoticias) # Crear el desplegable
        desplegableNoticias.grid(row=5, column=1, padx=(0, 20), sticky="w") # Ubicar el desplegable

        botonLeer = Button(self, text="Leer", command=lambda:Leer(valorMostradoNoticias, contenedorPreview)).grid(row=6, column=0, padx=(30, 20), pady=(10, 10), columnspan=2, sticky="ew") # Crear y ubicar el botón

        labelTOPNSeleccion = Label(self, text="TOP-N:").grid(row=7, column=0, sticky="w", padx=(30, 20)) # Crear y ubicar el texto
        opcionesTOPN = [2, 3, 4, 5, 6, 7, 8, 9, 10] # Opciones que se mostrarán en el desplegable del TOP-N
        valorMostradoTOPN = StringVar(self) # Para recoger el dato seleccionado
        valorMostradoTOPN.set(opcionesTOPN[3]) # Por defecto se mostrará y se escogerá la opción indicada en el enunciado
        desplegableTOPN = OptionMenu(self, valorMostradoTOPN, *opcionesTOPN) # Crear el desplegable
        desplegableTOPN.grid(row=7, column=1, padx=(0, 20), sticky="w") # Ubicar el desplegable

        labelFiltrar = Label(self, text="Mostrar de:").grid(row=8, column=0, sticky="w", padx=(30, 20)) # Crear y ubicar el texto
        opcionesFiltrar = ["Todos", "El Mundo", "El Pais", "20 Minutos"] # Opciones que se mostrarán en el desplegable de Filtrar
        valorMostradoFiltrar = StringVar(self) # Para recoger el dato seleccionado
        valorMostradoFiltrar.set(opcionesFiltrar[0]) # Por defecto se mostrará y se escogerá la opción indicada en el enunciado
        desplegableFiltrar = OptionMenu(self, valorMostradoFiltrar, *opcionesFiltrar) # Crear el desplegable
        desplegableFiltrar.grid(row=8, column=1, padx=(0, 20), sticky="w") # Ubicar el desplegable

        botonBuscar = Button(self, text="Buscar", command=lambda:BuscarCoseno(valorMostradoTOPN, valorMostradoFiltrar, contenedorPreview, valorMostradoRanking3, self)).grid(row=9, column=0, padx=(30, 20), pady=(10, 10), columnspan=2, sticky="ew") # Crear y ubicar el botón

        labelSeleccion = Label(self, text="Seleccionar noticia para leer:").grid(row=10,column=0, columnspan=4, sticky="w", padx=30, pady=(20, 20)) # Crear y ubicar el texto

        labelRanking = Label(self, text="Ranking:").grid(row=11, column=0, padx=(20, 0), pady=(20, 20), sticky="e") # Crear y ubicar el texto
        opcionesRanking = ["No hay noticias"] # Opciones que se mostrarán en el desplegable del ranking
        valorMostradoRanking3 = StringVar(self) # Para recoger el dato seleccionado
        valorMostradoRanking3.set("No hay noticias") # Por defecto se mostrará y se escogerá la opción indicada en el enunciado
        desplegableRanking = OptionMenu(self, valorMostradoRanking3, *["No hay noticias"]) # Crear el desplegable
        desplegableRanking.grid(row=11, column=1, padx=(0, 20), pady=(20, 20), sticky="w") # Ubicar el desplegable

        botonVerRecom = Button(self, text="Ver recomendacion", command=lambda:Leer(valorMostradoRanking3, contenedorNoticia)).grid(row=12, column=0, padx=(30, 20), pady=(10, 10), columnspan=2, sticky="ew") # Crear y ubicar el botón

        labelNoticia = Label(self, text="Texto de la noticia recomendada:").grid(row=1, column=3, sticky="sw", padx=20) # Crear y ubicar el texto
        contenedorNoticia = scrolledtext.ScrolledText(self, wrap="word", font=("Comic Sans", 9), width=50, height=35, padx=5) # Crear el cuadro de texto de varias filas
        contenedorNoticia.grid(row=2, column=3, rowspan=12, padx=(20, 20), pady=(0, 20)) # Ubicar el cuadro de texto de varias filas
        
        botonVolver = Button(self, text="Volver", command=lambda: master.cambiarFrame(VentanaPrincipal), bg="#FFFFCC").grid(row=13, column=0, columnspan=2, padx=(30, 20), pady=(10, 10))

## Gestión de la ventana principal y del cambio entre ventanas

In [14]:
 class Aplicacion(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        self._frame = None
        self.title("Sistemas inteligentes - Grupo 5")
        self.cambiarFrame(VentanaPrincipal)

    def cambiarFrame(self, frame_class):
        new_frame = frame_class(self)
        if self._frame is not None:
            self._frame.destroy()
        self._frame = new_frame
        self._frame.grid()

class VentanaPrincipal(tk.Frame):
    def __init__(self, master):
        tk.Frame.__init__(self, master)
        labelTitulo = tk.Label(self, text="Ventana Principal")
        labelTitulo.grid(row=0, column=0, padx=(60, 60), pady=(10, 10), sticky="ew")
        botonEtiquetas = tk.Button(self, text="Comparación por etiquetas: Sorensen-Dice", command=lambda: master.cambiarFrame(PantallaSorensen), bg="#DDDDFF")
        botonEtiquetas.grid(row=1, column=0, padx=(60, 60), pady=(10, 10), sticky="ew")
        botonQuery = tk.Button(self, text="Búsqueda por Query: Coseno", command=lambda: master.cambiarFrame(PantallaQuery), bg="#CCFFCC")
        botonQuery.grid(row=2, column=0, padx=(60, 60), pady=(10, 10), sticky="ew")
        botonComparacion = tk.Button(self, text="Comparación por textos: Coseno", command=lambda: master.cambiarFrame(PantallaComparacion), bg="#FFFFCC")
        botonComparacion.grid(row=3, column=0, padx=(60, 60), pady=(10, 10), sticky="ew")
        labelSeparador = tk.Label(self, text="-"*60)
        labelSeparador.grid(row=4, column=0, padx=(60, 60), pady=(10, 10), sticky="ew")     
        botonSalir = Button(self, text="Salir", command=master.destroy)
        botonSalir.grid(row=6, column=0, padx=(60, 60), pady=(10, 20), sticky="ew")

if __name__ == "__main__":
    app = Aplicacion()
    app.mainloop()